<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/RNA_Workshop/Explore_Nell2_Pathways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy --quiet
!pip install pybiomart --quiet
!pip install python-igraph --quiet
!pip install louvain --quiet
!pip install pynndescent --quiet
# !pip install keggtools --quiet

!pip install scipy

In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd
import pybiomart
from tqdm import tqdm
import urllib.request
from IPython.display import clear_output
from matplotlib.pyplot import rc_context
import scipy
import logging

In [ ]:
def init_GO():
    # !pip install goatools --quiet


    '''Get Gene Lists and metadata from ncbi'''
    import os
    import urllib.request
    gene_list_url='https://github.com/dtabuena/Resources/raw/1e3f0ef18ba127b71a9e6b93f7624e3a28fe87c1/GO%20Files/gene_result.txt'
    urllib.request.urlretrieve(gene_list_url, 'gene_result.txt')
    scripts_path = [p for p in os.environ['PATH'].split(';') if 'Scripts' in p][0]
    ncbi_path = os.path.join(scripts_path,'ncbi_gene_results_to_python.py')
    !python $ncbi_path -o genes_ncbi_mus_musculus_proteincoding.py gene_result.txt
    from genes_ncbi_mus_musculus_proteincoding import GENEID2NT as GeneID2nt_mus



    '''Get Key Funcs'''
    from goatools.base import download_go_basic_obo
    from goatools.base import download_ncbi_associations
    from goatools.obo_parser import GODag
    from goatools.anno.genetogo_reader import Gene2GoReader
    from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

    '''Download Current Go Annotations'''
    obo_fname = download_go_basic_obo()
    fin_gene2go = download_ncbi_associations()
    obodag = GODag("go-basic.obo")




    '''Get Mapper from Symbol to Gene and Inv'''
    mapper = {}
    for key in GeneID2nt_mus:
        mapper[GeneID2nt_mus[key].Symbol] = GeneID2nt_mus[key].GeneID
    inv_map = {v: k for k, v in mapper.items()}



    '''Read NCBI's gene2go. Store annotations in a list of namedtuples '''
    objanno = Gene2GoReader(fin_gene2go, taxids=[10090])
    # Get namespace2association where:
    #    namespace is:
    #        BP: biological_process
    #        MF: molecular_function
    #        CC: cellular_component
    #    assocation is a dict:
    #        key: NCBI GeneID
    #        value: A set of GO IDs associated with that gene
    ns2assoc = objanno.get_ns2assc()


    '''Create a GO Object'''
    goeaobj = GOEnrichmentStudyNS(
            GeneID2nt_mus.keys(), # List of mouse protein-coding genes
            ns2assoc, # geneid/GO associations
            obodag, # Ontologies
            propagate_counts = False,
            alpha = 0.05, # default significance cut-off
            methods = ['fdr_bh']) # defult multipletest correction method


    ''' PASS '''
    num_terms = {}
    GO_items = []
    temp = goeaobj.ns2objgoea['BP'].assoc
    num_terms['biological_process']=len(temp)
    for item in temp:
        GO_items += temp[item]
    temp = goeaobj.ns2objgoea['CC'].assoc
    num_terms['cellular_component']=len(temp)
    for item in temp:
        GO_items += temp[item]
    temp = goeaobj.ns2objgoea['MF'].assoc
    num_terms['molecular_function']=len(temp)
    for item in temp:
        GO_items += temp[item]


    # GO_items = goeaobj.ns2objgoea['BP'].assoc + goeaobj.ns2objgoea['CC'].assoc + goeaobj.ns2objgoea['MF'].assoc

    # def go_it(test_genes):
    #     ''' Quick Access Function for doing the GO associations '''
    #     logging.info(f'input genes: {len(test_genes)}')
    #     mapped_genes = []
    #     for gene in test_genes:
    #         try:
    #             mapped_genes.append(mapper[gene])
    #         except:
    #             pass
    #     logging.info(f'mapped genes: {len(mapped_genes)}')
    #     goea_results_all = goeaobj.run_study(mapped_genes)
    #     goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    #     GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, x.p_uncorrected, x.p_fdr_bh,\
    #                 x.ratio_in_study[0], x.ratio_in_study[1], GO_items.count(x.GO), list(map(lambda y: inv_map[y], x.study_items)),\
    #                 ], goea_results_sig)), columns = ['GO', 'term', 'class', 'p', 'p_corr', 'n_genes',\
    #                                                     'n_study', 'n_go', 'study_genes'])
    #     GO = GO[GO.n_genes > 1]
    #     return GO

    def go_it(test_genes):
        ''' Quick Access Function for doing the GO associations '''
        logging.info(f'input genes: {len(test_genes)}')
        mapped_genes = []
        for gene in test_genes:
            try:
                mapped_genes.append(mapper[gene])
            except:
                pass
        logging.info(f'mapped genes: {len(mapped_genes)}')
        goea_results_all = goeaobj.run_study(mapped_genes)
        goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
        GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, num_terms[x.goterm.namespace], x.p_uncorrected, x.p_fdr_bh,\
                    x.ratio_in_study[0], x.ratio_in_study[1], GO_items.count(x.GO), list(map(lambda y: inv_map[y], x.study_items)),\
                    ], goea_results_sig)), columns = ['GO', 'term', 'class', 'class_size', 'p', 'p_corr', 'n_genes',\
                                                        'n_study', 'n_go', 'study_genes'])
        GO = GO[GO.n_genes > 1]

        GO['enrich_ratio'] = ( GO['n_genes']/GO['n_study'] ) / ( GO['n_go']/GO['class_size'] )

        return GO

    return go_it, num_terms

In [ ]:
def trim_key(k):
    floxed_dict = {'GSM5106175_YH_KZ03_01':('E3fKI_Syn_Cre602_15m','GSM5106175_602_E3fKI_15_XX'),
                   'GSM5106176_YH_KZ03_03':('E4fKI_Syn_Cre475_15m','GSM5106176_475_E4fKI_15_XX')}
    for f in floxed_dict.keys():
        if f in k: return floxed_dict[f][1]
    k = k.replace('_raw_gene_bc_matrices_h5.h5',"")
    return k
def query_capitilaziation(gene,adata):
    try:
        return adata.var.index[ [g.lower() for g in list(adata.var.index)].index(gene.lower()) ]
    except:
        return gene + ' not_found'
def z_score(x,axis=-1):
    x=np.array(x)
    return (x-np.mean(x,axis=axis))/np.std(x,axis=axis)



In [ ]:
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Seurat_Scanpy/Scanpy_data/")
os.listdir()

In [ ]:
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Seurat_Scanpy/Scanpy_data/")
adata = sc.read_h5ad('./2023_11_07_KZ_anndata.h5ad')

In [ ]:
display(adata.obs.head())

meta_df = pd.read_csv('./kz_metadata.csv').set_index('Barcodes')

adata_meta= adata.copy()
adata_meta.obs["Cluster_ID"]=np.nan
adata_meta.obs["Cluster_ID"]= meta_df["Cluster_ID"]
display(adata_meta.obs.head())

all_cats = list(set(meta_df["Cluster_ID"]))
print(all_cats)

In [ ]:
########### Sub Divide Clusters of Interest
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/_Nell2_enrichment/")
# dgc_01_adata =  adata_meta[adata_meta.obs["Cluster_ID"] == '01 Dentate Gyrus Granule Cells'].copy()
# display(dgc_01_adata)
dgc_02_adata =  adata_meta[adata_meta.obs["Cluster_ID"] == '02 Dentate Gyrus Granule Cells'].copy()
display(dgc_02_adata)
CA3_06_adata =  adata_meta[adata_meta.obs["Cluster_ID"] == '06 CA2/CA3 Pyramids'].copy()
display(CA3_06_adata)


In [ ]:
def find_corr_genes(adata,gene,pct=1,to_plot=True):
    X_ = adata.X.toarray()
    gene_ind = list(adata.var['name']).index(gene)
    target_row = X_[:,gene_ind]
    num_genes= X_.shape[1]
    gene_corr = np.ones([num_genes])
    p_vals = np.zeros([num_genes])
    for g in range(num_genes):
        gene_corr[g],p_vals[g] = scipy.stats.pearsonr( target_row,X_[:,g])

    gene_rank =np.argsort(np.argsort(-gene_corr))
    gene_corr_plot=gene_corr.copy()
    gene_corr_plot[gene_ind]=np.nan

    low,high = np.nanpercentile(gene_corr,[pct,100-pct])

    high_bool = gene_corr>high
    low_bool = gene_corr<low

    high_names=adata.var['name'][high_bool]
    low_names=adata.var['name'][low_bool]


    if to_plot:
        # fig,ax=plt.subplots(1,2,figsize=(3,1),width_ratios=(5,1),dpi=300)
        fig,ax=plt.subplots(1,1,figsize=(3,1),dpi=300)
        ax=[ax]
        ax[0].scatter(gene_rank,gene_corr_plot,s=1)

        ax[0].axhline(high,color='k',linewidth=1)
        ax[0].axhline(low,color='k',linewidth=1)

        ax[0].axhline(high,color='k',linewidth=1)
        ax[0].axhline(low,color='k',linewidth=1)



    ax[0].set_xlabel('Rank')
    ax[0].set_ylabel('Pearson R')
    plt.show()


    FDR_p_vals = p_vals*np.nan
    results_dict = {'gene_corr':gene_corr, 'high_bool':high_bool,'low_bool':low_bool ,'high_names':high_names,'low_names':low_names,'gene_rank':gene_rank}
    results_dict['table']= pd.DataFrame({'gene_name':adata.var['name'],
                                         'gene_rank':gene_rank,
                                         'gene_corr':gene_corr,
                                         'p_vals':p_vals,
                                         'FDR_p_vals':FDR_p_vals,
                                         'high_bool':high_bool,
                                         'low_bool':low_bool}).set_index('gene_name').sort_values('gene_rank',axis=0)

    return results_dict, fig, ax

pct = 5
dgc_02_nell_results,_,_ = find_corr_genes(dgc_02_adata,'Nell2',pct=pct)
dgc_02_apoe_results,_,_ = find_corr_genes(dgc_02_adata,'Apoe',pct=pct)

CA3_06_nell_results,_,_ = find_corr_genes(CA3_06_adata,'Nell2',pct=pct)
CA3_06_apoe_results,_,_ = find_corr_genes(CA3_06_adata,'Apoe',pct=pct)


In [ ]:

def write_tables(res,prefix):
    high_table = res['table']
    high_table = high_table[high_table['high_bool']]
    high_table.to_csv(prefix+'_high_corr_names.csv')

    low_table = CA3_06_nell_results['table']
    low_table = high_table[low_table['low_bool']]
    low_table.to_csv(prefix+'_low_corr_names.csv')
    return None

os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/_Nell2_enrichment/")
write_tables(CA3_06_nell_results,'CA3_06_nell')
write_tables(CA3_06_apoe_results,'CA3_06_apoe')
write_tables(dgc_02_nell_results,'dgc_02_nell')
write_tables(dgc_02_apoe_results,'dgc_02_apoe')



In [ ]:
go_it,  num_terms = init_GO()


In [ ]:
logging.basicConfig(filename='example.log',level=logging.DEBUG)
def get_onto_from_dict(dict_results):
    h = len(dict_results['high_names'])
    l = len(dict_results['low_names'])
    logging.info(f"{h} high genes and {l} low genes")
    dict_results['go_high'] = go_it(dict_results['high_names'])

    dict_results['go_low'] = go_it(dict_results['low_names'])
    return dict_results


CA3_06_nell_results = get_onto_from_dict(CA3_06_nell_results)
CA3_06_apoe_results = get_onto_from_dict(CA3_06_apoe_results)
dgc_02_nell_results = get_onto_from_dict(dgc_02_nell_results)
dgc_02_apoe_results = get_onto_from_dict(dgc_02_apoe_results)



In [ ]:
go_group_dict={'CA3_nell_h':CA3_06_nell_results['go_high'],
               'CA3_apoe_h':CA3_06_apoe_results['go_high'],
               'dgc_nell_h':dgc_02_nell_results['go_high'],
               'dgc_apoe_h':dgc_02_apoe_results['go_high']}
for k,v in go_group_dict.items():
    go_group_dict[k] = [g for i,g in enumerate(v['GO']) if 'biological_process' in v['class'][i] ]

In [ ]:
go_group_dict

In [ ]:
''' Analyze High Corr Nell in DG vs CA3, Process only '''
def get_go_sets( go_group_dict , exclude_all_none=True):
    fig,ax=plt.subplots(2,2,figsize=(3,3),width_ratios=[5,1.5],height_ratios=[5,2],dpi=300)
    null_ax = ax[0,1]
    null_ax.axis('off')
    combo_ax = ax[1,0]
    overlap_ax = ax[0,0]
    set_size_ax = ax[1,1]


    groups = list(go_group_dict.keys())
    combos = list(itertools.product([True,False], repeat=len(groups)))
    combo_sum = [np.sum(c) for c in combos]
    combos=np.array(combos)[np.argsort(combo_sum)]
    if exclude_all_none:
        combos = [c for c in combos if c.any()]
    true_xy = np.where(combos)
    false_xy = np.where(np.logical_not(combos))
    dot_size=12
    combo_ax.scatter(true_xy[0],true_xy[1],color='k',s=dot_size)
    combo_ax.scatter(false_xy[0],false_xy[1],color='lightgrey',s=dot_size)
    combo_ax.set_yticks(range(4),groups)
    combo_ax.set_xticks([])
    combo_ax.set_ylim([-.5,len(groups)-.5])
    my_map = np.cumsum(np.ones_like(combos),axis=1)*combos
    for row,vals in enumerate(my_map):
        vals_nz = [v for v in vals if v > 0]
        if np.sum(vals>0)>1:
            combo_ax.plot([row,row],[np.min(vals_nz)-1,np.max(vals_nz)-1],color='k',linewidth=.5 )



    all_gos = [x for v in go_group_dict.values() for x in v] #get dict values and flatten
    all_go_table = []
    for go in all_gos:
        all_go_table.append([go in v for v in go_group_dict.values()])
    all_go_table=np.array(all_go_table)
    index_combos = {i:c for i,c in enumerate(combos)}
    combos_index = {str(v):k for k,v in index_combos.items()}
    all_go_combo_indicies = [combos_index[str(g)] for g in all_go_table]
    hist,edges = np.histogram(all_go_combo_indicies, bins=len(combos), range=[0,len(combos_index)])
    overlap_ax.bar(edges[:-1],hist,color='k')
    overlap_ax.set_xlim(combo_ax.get_xlim() )
    overlap_ax.set_xticks([])
    overlap_ax.set_ylabel('Intersection (#)')



    set_size_ax.barh( list(go_group_dict.keys()), [len(v) for v in go_group_dict.values()],color='k'   )
    # set_size_ax.invert_xaxis()
    set_size_ax.set_yticks([])
    set_size_ax.set_xlabel('Group Size (#)')

    # plt.tight_layout()
    return combos,all_go_table

combos, all_go_table =  get_go_sets(go_group_dict)

In [ ]:
combos = [c for c in combos if c.any()]
print(combos)


In [ ]:
def upset_plot(group_dict, figsize=(3,3), exclude_all_none=True):

    'group_dict->dict with keys=categories/groups an values=list(members)'

    fig,ax=plt.subplots(2,2,figsize=figsize,width_ratios=[5,1.5],height_ratios=[5,2],dpi=300)
    null_ax = ax[0,1]
    null_ax.axis('off')
    combo_ax = ax[1,0]
    overlap_ax = ax[0,0]
    set_size_ax = ax[1,1]


    group_names = list(group_dict.keys())
    possible_combos = list(itertools.product([True,False], repeat=len(group_names)))

    combo_sum = [np.sum(c) for c in possible_combos]
    possible_combos=np.array(possible_combos)[np.argsort(combo_sum)]
    if exclude_all_none: #exclude members that occur on no list
        possible_combos = [c for c in possible_combos if c.any()]

    true_xy = np.where(possible_combos)
    false_xy = np.where(np.logical_not(possible_combos))
    dot_size=12
    combo_ax.scatter(true_xy[0],true_xy[1],color='k',s=dot_size)
    combo_ax.scatter(false_xy[0],false_xy[1],color='lightgrey',s=dot_size)
    combo_ax.set_yticks(range(4),group_names)
    combo_ax.set_xticks([])
    combo_ax.set_ylim([-.5,len(group_names)-.5])
    my_map = np.cumsum(np.ones_like(possible_combos),axis=1)*combos
    for row,vals in enumerate(my_map):
        vals_nz = [v for v in vals if v > 0]
        if np.sum(vals>0)>1:
            combo_ax.plot([row,row],[np.min(vals_nz)-1,np.max(vals_nz)-1],color='k',linewidth=.5 )



    all_members = [x for v in group_dict.values() for x in v] #get dict values and flatten
    all_members_table = []
    for m in all_members:
        all_members_table.append([m in v for v in group_dict.values()])

    all_members_table=np.array(all_members_table)
    index_combos = {i:c for i,c in enumerate(possible_combos)}
    combos_index = {str(v):k for k,v in index_combos.items()}

    all_members_combo_indicies = [combos_index[str(g)] for g in all_members_table]
    hist,edges = np.histogram(all_members_combo_indicies, bins=len(possible_combos), range=[0,len(combos_index)])
    overlap_ax.bar(edges[:-1],hist,color='k')
    overlap_ax.set_xlim(combo_ax.get_xlim() )
    overlap_ax.set_xticks([])
    overlap_ax.set_ylabel('Intersection (#)')



    set_size_ax.barh( list(group_dict.keys()), [len(v) for v in group_dict.values()],color='k'   )
    set_size_ax.set_yticks([])
    set_size_ax.set_xlabel('Group Size (#)')

    return fig,ax


fig,ax = upset_plot(go_group_dict,figsize=(4,2.5))